<a href="https://colab.research.google.com/github/cbonnin88/RailFlow/blob/main/Railflow_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
!pip install streamlit
!pip install -q streamlit plotly scikit-learn joblib
!pip install -q pyngrok

In [90]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import plotly.express as px
import datetime
import os


# --- CONFIGURATION ---
st.set_page_config(page_title='RailFlow Predictor', page_icon='🚄', layout='wide')

# -- LOAD BRAIN ---
@st.cache_resource(show_spinner=False)
def load_data():
    # 1. Load the file once
    artifacts = joblib.load("railflow_brain.joblib")

    # 2. Extract the objects
    model = artifacts["model"]
    le_origin = artifacts["le_origin"]
    le_dest = artifacts["le_dest"]

    # 3. PRE-COMPUTE the lists here (so we don't do it on every click)
    # We strip whitespace and sort them to look professional
    origin_list = sorted([str(x).strip() for x in le_origin.classes_])
    dest_list = sorted([str(x).strip() for x in le_dest.classes_])

    # 4. OPTIMIZATION: If list is massive (>500), slice it for performance
    if len(origin_list) > 500:
        origin_list = origin_list[:500]
    if len(dest_list) > 500:
        dest_list = dest_list[:500]

    return model, le_origin, le_dest, origin_list, dest_list

with st.spinner('🧠 Loading AI Model & Station Data...'):
    try:
        # Unpack all 5 variables
        model, le_origin, le_dest, origin_list, dest_list = load_data()
    except FileNotFoundError:
        st.error("⚠️ Model file not found!")
        st.stop()

# --- SIDEBAR: INPUTS ---
st.sidebar.header("🔍 Search Parameters")

# 1. Dropdowns
default_origin_ix = origin_list.index('Paris Gare de Lyon') if 'Paris Gare de Lyon' in origin_list else 0
default_dest_ix = dest_list.index('Lyon Part-Dieu') if 'Lyon Part-Dieu' in dest_list else 1

origin = st.sidebar.selectbox("Origin Station", origin_list, index=default_origin_ix)
destination = st.sidebar.selectbox("Destination Station", dest_list, index=default_dest_ix)

# 2. Date & Time
travel_date = st.sidebar.date_input('Travel Date',datetime.date.today() + datetime.timedelta(days=7))
travel_time = st.sidebar.time_input('Departure Time', datetime.time(9,0))

# 3. Calculate Derived Features
search_date = datetime.date.today()
lead_time_days = (travel_date - search_date).days
day_of_week = travel_date.weekday() # Monday is 0, Sunday is 6
hour = travel_time.hour


# --- MAIN PANEL ---
st.title('🚄 RailFlow Conversion Predictor')
st.markdown(f"""
**Scenario:** User searching for **{origin}** → **{destination}**
- **Lead Time:** {lead_time_days} days in advance
- **Departure Hour:** {hour}:00
""")

if st.button('🔮 Predict Booking Probability', type='primary'):
  # 1. Encode Inputs
  try:
    origin_code = le_origin.transform([origin])[0]
    dest_code = le_dest.transform([destination])[0]
  except ValueError:
    st.error('Unknown station selected')
    st.stop()

  # 2. Prepare Data for Model
  input_data = pd.DataFrame({
      'origin_code': [origin_code],
      'dest_code': [dest_code],
      'search_day_of_week': [day_of_week],
      'search_hour': [hour],
      'lead_time_days': [lead_time_days]
  })

  # 3. Predict
  probability = model.predict_proba(input_data)[0][1] # Probability of Class 1 (Booking)

  # --- VISUALIZATION OF RESULT ---
  st.divider()
  col1, col2 = st.columns([2,1])

  with col1:
    st.subheader('Prediction Results')
    st.metric(label="Probability to Book", value=f"{probability:.1%}", delta=f"{'High Intent' if probability > 0.7 else 'Low Intent'}")

    # Action Logic
    if probability > 0.7:
      st.success('✅ **High Intent:** User likely to book organically.')
    elif probability > 0.3:
      st.warning('⚡ **Medium Intent:** Recommend sending a **5% Discount Push Notification**.')
    else:
      st.error('🛑 **Low Intent:** User is likely just browsing.')

    with col2:
      # Gauge Chart (Donut)
      color = "#00C851" if probability > 0.7 else "#ffbb33" if probability > 0.3 else "#ff4444"
      fig_doughnut = px.pie(
          values=[probability, 1-probability],
          names=['Likely to Book', 'Not Likely to Book'],
          hole=0.7,
          color_discrete_sequence = [color,'#e0e0e0']
      )
      fig_doughnut.update_layout(showlegend=False, annotations=[dict(text=f'{int(probability*100)}%', x=0.5,y=0.5,font_size=40,showarrow=False)])
      st.plotly_chart(fig_doughnut,use_container_width=True)

    # --- EXPLAINABILITY (Why?) ---
    st.divider()
    st.subheader('Why this prediction')
    st.caption('Feature importance based on the Random Forest Model')

    importances = pd.DataFrame({
        'Factor':['Route (Origin)','Route (Dest)','Day of Week','Time of Day','Lead Time'],
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=True)


    fig_imp = px.bar(
        importances,
        x='Importance',
        y='Factor',
        orientation='h',
        title='Model Factors'
    )
    fig_imp.update_layout(margin=dict(t=0,b=0,l=0,r=0),height=300)
    st.plotly_chart(fig_imp, use_container_width=True)

Overwriting app.py


In [91]:
# 1. Verify Model Exists
if not os.path.exists("railflow_brain.joblib"):
    print("❌ ERROR: 'railflow_brain.joblib' not found.")
    print("👉 Please re-run the 'Export Model' step from Phase 6 in this notebook.")
else:
    print("✅ Model file found. Ready to go!")

✅ Model file found. Ready to go!


In [92]:
!streamlit run app.py &>/dev/null &

In [93]:
from pyngrok import ngrok

# Set the ngrok authentication token for the current session
ngrok.set_auth_token('38YtXAE5tAqhAkvxTX0HYHqQzTM_2DqLmnAxDAmEzCY4h1ztZ')

tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

In [94]:
public_url = ngrok.connect(8501).public_url
print(f"🚀 Your Stable App URL: {public_url}")

🚀 Your Stable App URL: https://378965203199.ngrok-free.app
